In [ ]:
# -*- coding: utf-8 -*-
from scipy.signal import savgol_filter
import itertools
import os
from osgeo import gdal
import numpy as np
import scipy.stats, scipy.signal
import numpy.ma as ma
from joblib import Parallel, delayed
import pandas as pd
import scipy.stats
import datetime
import warnings
import rasterio
warnings.filterwarnings('ignore')

In [ ]:
### Pre-processing modified from the method of Chen et al. (2004): https://www.sciencedirect.com/science/article/abs/pii/S003442570400080X
def clean_ndvi_timeseries(orig_input):
    """Following Chen et al. 2004"""
    # Remove impossible values, linear fit
    roll_ma = orig_input.rolling(window='20D').apply(np.nanmax)
    roll_mi = orig_input.rolling(window='20D').apply(np.nanmin)

    diff = roll_ma - roll_mi
    orig_input[diff > 0.4] = np.nan

    # Remove remaining missed clouds/water
    orig_input[orig_input < 0] = np.nan

    # Mask out non-vegetated areas
    mns = []
    for yr in orig_input.groupby(orig_input.index.year):
        yrdata = yr[1]
        mns.append(np.nanmean(yrdata.values))

    if np.nanmean(mns) < 0.1:
        return np.array((np.nan,) * orig_input.shape[0]), False

    # STEP 1 - Do the interpolation
    else:
        data = orig_input.interpolate('linear').bfill().ffill()
        N_0 = data.copy()

        # STEP 2 - Fit SG Filter, using best fitting parameters
        arr = np.empty((4, 3))
        for m, d in list(itertools.product([4, 5, 6, 7], [2, 3, 4])):
            data_smoothed = pd.Series(savgol_filter(data.values, window_length=2 * m + 1, polyorder=d, deriv=0, delta=1.0), index=data.index)
            err = np.nansum(data_smoothed - data) ** 2
            arr[m - 4, d - 2] = err
        m, d = np.where(arr == np.nanmin(arr))
        m, d = m[0] + 4, d[0] + 2
        data_smoothed = pd.Series(savgol_filter(data.values, window_length=2 * m + 1, polyorder=d, deriv=0, delta=1.0), index=data.index)
        diff = N_0 - data_smoothed

        # STEP 3 - Create weights array based on difference from curve
        max_dif = np.nanmax(np.abs(diff.values))
        weights = np.zeros(np.array(data_smoothed.values).shape)
        weights[data_smoothed >= N_0] = 1
        weights[data_smoothed < N_0] = 1 - (np.abs(diff.values) / max_dif)[data_smoothed < N_0]

        # STEP 4 - Replace values that were smoothed downwards with original values
        data_smoothed[N_0 >= data_smoothed] = N_0[N_0 >= data_smoothed]
        data_smoothed[N_0 < data_smoothed] = data_smoothed[N_0 < data_smoothed]

        # STEP 5 - Resmooth with different parameters
        data_fixed = pd.Series(savgol_filter(data.values, window_length=9, polyorder=6, deriv=0, delta=1.0), index=data.index)

        # STEP 6 - Calculate the fitting effect
        Fe_0 = np.nansum(np.abs(data_fixed.values - N_0.values) * weights)

        data = data_fixed.copy()

        count = 0
        # while Fe_0 >= Fe and Fe <= Fe_1:
        # while Fe_0 >= Fe:
        while count < 5:  # Faster and just as effective to limit the number of loops rather than having a convergence factor
            # STEP 4 - Replace values that were smoothed downwards with original values
            data[N_0 >= data] = N_0[N_0 >= data]
            data[N_0 < data] = data[N_0 < data]

            # STEP 5 - Resmooth with different parameters
            data_fixed = pd.Series(savgol_filter(data.values, window_length=9, polyorder=6, deriv=0, delta=1.0), index=data.index)
            # data_fixed = pd.Series(savitzky_golay(data.values, 5, 2, deriv=0, rate=1), index=data.index)

            # STEP 6 - Calculate the fitting effect
            # Fe = np.nansum(np.abs(data_fixed.values - N_0.values) * weights)
            # data = data_fixed.copy()
            # Fe_0 = Fe.copy()
            count += 1
            # if Fe <= Fe_0:
            # data_fixed = data_fixed_smoothed.copy()
            # data_fixed[N_0 >= data_fixed_smoothed] = N_0[N_0 >= data_fixed_smoothed]
            # data_fixed[N_0 < data_fixed_smoothed] = data_fixed_smoothed[N_0 < data_fixed_smoothed]
            # if Fe > Fe_0:
            #    break
        return np.array(data.values), True

In [ ]:
### Detrend and Deseason
def harmonic_fit(ser, order=3):
    import statsmodels.api as sm
    harm_freq = list(range(1, order + 1))

    x, y = ser.index, ser.values
    x = pd.to_datetime(x, format='%Y%m%d')
    x = np.array([(x - pd.Timestamp('2000-01-01')).days for x in x]) / 365.25
    x_rad = x * 2 * np.pi  # Convert days to radians for harmonic fitting

    # Create empty array to hold the independents
    nr_indep = order * 2 + 2
    indep = np.empty((y.shape[0], nr_indep))

    # Add constant for intercept and then time
    indep[:, 0] = 1
    indep[:, 1] = x_rad

    # Now create the harmonic variables
    i = 2
    for freq in harm_freq:
        cos = np.cos(x_rad * freq)
        sin = np.sin(x_rad * freq)
        indep[:, i] = cos
        i = i + 1
        indep[:, i] = sin
        i = i + 1

    model = sm.OLS(y, indep, missing='drop').fit()
    coefs = model.params
    fitted = []
    for t in range(x_rad.shape[0]):
        data = indep[t, :]
        harm_term = np.nansum(coefs * data)
        fitted.append(harm_term)
    fitted = np.array(fitted)
    return pd.Series(ser.values - fitted, index=ser.index)


def runmean(x, w):
    n = x.shape[0]
    xs = np.zeros_like(x)
    for i in range(w // 2):
        xs[i] = np.nanmean(x[: i + w // 2 + 1])
    for i in range(n - w // 2, n):
        xs[i] = np.nanmean(x[i - w // 2 + 1:])
    for i in range(w // 2, n - w // 2):
        xs[i] = np.nanmean(x[i - w // 2: i + w // 2 + 1])
    return x - xs


def deseason_detrend(ser, yrs, yl):
    rm_offline = pd.Series(runmean(ser.values, yrs*yl), index=ser.index)
    deseason_rolling = harmonic_fit(rm_offline, order=3)
    return deseason_rolling.values

In [ ]:
### Cauculate theoretical recovery rates from AC1/variance
def calc_ar1(x):
    # return np.corrcoef(x[:-1], x[1:])[0,1]
    return ma.corrcoef(ma.masked_invalid(x[:-1]), ma.masked_invalid(x[1:]))[0, 1]


def compute_lam(x, dt):
    dx = (x[1:] - x[:-1]) / dt
    return scipy.stats.linregress(x[:-1], dx)[0]


def compute_sigma(x, dt=1):
    dx = (x[1:] - x[:-1]) / dt
    lamb = compute_lam(x, dt)
    diff = dx - lamb * x[:-1]
    return np.std(diff) * np.sqrt(dt)


def check_ts(x):
    v = np.nanvar(x)
    ar1 = calc_ar1(x)
    lambda_ar1 = np.log(ar1)
    sigma = compute_sigma(x)
    lambda_var = -sigma ** 2 / (2 * v)

    return lambda_ar1, lambda_var